In [ ]:
from Peak_Search_Linien import ini_peak_search, reso_det, mod_peak_search,zero_measure,fir_measure
from windfreak import SynthHD
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from linien_utils import get_waveform, set_scan_range
from linien_client.connection import LinienClient

import h5py
import time
import tqdm
import csv

In [ ]:
resource_name_windfreak = "COM8"
synthd = SynthHD(resource_name_windfreak)
synthd.init()
rf_out = 0

In [ ]:
c = LinienClient(
    host="10.10.11.30",
    user="root",
    password="root"
)
c.connect(autostart_server=True, use_parameter_cache=True)


Peak Search

In [ ]:
synthd[rf_out].enable = False
zero_amp_all = []
fir_amp_all = []
pow_max_all = []
scan_data = []
power_scan = np.linspace(-15, 8, 10)
fre_scan = np.linspace(1700e6,2700e6,1)

fir_amps = []

for rf_fre in fre_scan:
    synthd[rf_out].enable = False
    ini_rev_data = get_waveform(c)
    ini_peak = ini_peak_search(ini_rev_data)
    ini_reso = reso_det(ini_peak)
    zero_amp = []
    fir_amp = []
    synthd[rf_out].enable = True
    for shdpower in tqdm.tqdm(power_scan):
        synthd[rf_out].frequency = rf_fre
        synthd[rf_out].power = shdpower
        mod_rev_data  = get_waveform(c)
        # time.sleep(1)
        mod_peak = mod_peak_search(mod_rev_data)
        mod_zero_peak1, mod_zero_peak2, mod_zero_amp = zero_measure(mod_rev_data, ini_peak, mod_peak, ini_reso)
        mod_fir_peak, mod_fir_amp = fir_measure(c, mod_rev_data, ini_peak, mod_peak, rf_fre,ini_reso)
        
        if mod_fir_peak == mod_zero_peak2:
            mode_fir_amp = 0.05
        zero_amp.append(mod_zero_amp)
        fir_amp.append(mod_fir_amp)
        # time.sleep(1)
    pow_max = power_scan[fir_amp.index(max(fir_amp))]
    scan_data.append([rf_fre,pow_max])
    fir_amps.append(fir_amp)
    pow_max_all.append(pow_max)
    zero_amp_all.append(zero_amp)
    fir_amp_all.append(fir_amp)
    